In [ ]:
# Set a range of dayobs values to search - 
day_obs_min = "2024-11-16"
day_obs_max = "2024-11-16"

# HEY NOTE: script salIndexes are reused (for different purposes) when scriptqueue is restarted
# script salindex is not unique over all periods of time

# also would like:
# identifier for the scripts to replace or supplement script salIndex so that it's unique (hash for script)
# Recording of hash for ts_config_ocs or for block itself --- this may be present in new obs_env (check manage_obs_env??)

# EFD Scripts + Logs 

In [ ]:
#import lsst.ts.xml.enums.GeneralState
#help(ts_xml_enums)
#import lsst.summit.utils
#help(lsst.summit.utils)

In [ ]:
# from https://github.com/lsst-ts/ts_xml/blob/develop/python/lsst/ts/xml/enums -- import would be better 
# Informational reference point .. 
import enum

class GeneralState(enum.IntEnum):
    """CSC summaryState constants."""

    OFFLINE = 4
    STANDBY = 5
    DISABLED = 1
    ENABLED = 2
    FAULT = 3


class ScriptProcessState(enum.IntEnum):
    """ScriptQueue script.processState event constants."""

    UNKNOWN = 0
    LOADING = 1
    CONFIGURED = 2
    RUNNING = 3
    DONE = 4
    LOADFAILED = 5
    CONFIGURE_FAILED = 6
    TERMINATED = 7
    CONFIGUREFAILED = 6  # deprecated alias for CONFIGURE_FAILED

class ScriptState(enum.IntEnum):
    """ScriptState constants."""

    UNKNOWN = 0
    UNCONFIGURED = 1
    CONFIGURED = 2
    RUNNING = 3
    PAUSED = 4
    ENDING = 5
    STOPPING = 6
    FAILING = 7
    DONE = 8
    STOPPED = 9
    FAILED = 10
    CONFIGURE_FAILED = 11

# status_dict = dict((e.name, e.value) for e in GeneralState)
# process_state_dict = dict((e.name, e.value) for e in ScriptProcessState)
# script_state_dict = dict((e.name, e.value) for e in ScriptState)

# display(f"General state enum values: {status_dict}")
# display(f"Script State enum values: {script_state_dict}")
# display(f"Script Process State enum values: {process_state_dict}")

In [ ]:
import numpy as np
import pandas as pd
from pandas import option_context
from IPython.display import display, Markdown, HTML
from astropy.time import Time, TimeDelta
import astropy.units as u

from lsst_efd_client import EfdClient

import matplotlib.pyplot as plt

In [ ]:
# Connect to the EFD 

# efd_client = EfdClient('summit_efd') 
efd_client = EfdClient('usdf_efd')

In [ ]:
# Set a range of dayobs values to search - 
t_start = Time(f"{day_obs_min}T12:00:00", format='isot', scale='utc')
#t_start = t_start + TimeDelta(0.45, format='jd')
#t_start = Time('2024-11-06T07:19:15.397500', format='isot', scale='utc') - TimeDelta(1/24, format='jd')

t_end = Time(f"{day_obs_max}T12:00:00", format='isot', scale='utc') + TimeDelta(1, format='jd')
#t_end = t_start + TimeDelta(2/24, format='jd')
print(f"Querying the EFD from {t_start.iso} to {t_end.iso}")

In [ ]:
# Query any EFD topic for the timespan day_obs_min to day_obs_max, when you don't already know the fields
# topic = lsst.sal.ScriptQueue.command_add
# fields = await efd_client.get_fields(topic)
# fields = [f for f in fields if 'private' not in f and f != 'name' and f!= "duration"]
# dd = await efd_client.select_time_series(topic, fields, tstart, tend)
# or top 5 .. 
# dd = await efd_client.select_top_n(topic, fields, 5)

In [ ]:
# Some utilities 
def apply_enum(x, column, enumvals):
    return enumvals(x[column]).name

def convert_index_to_time(x):
    return Time(x.name)

def usdf_requests(API_ENDPOINT, params, try_dev=True):
    # Try twice
    response = requests.get(API_ENDPOINT, params)
    if response.status_code != 200:
        response = requests.get(API_ENDPOINT, params)
    # Could dev as backup if still not working .. 
    # although this may not be the same content? 
    if try_dev:
        if response.status_code != 200:
            API_DEV = API_ENDPOINT.replace('usdf-rsp', 'usdf-rsp-dev')
            # Try twice
            response = requests.get(API_DEV, params)
            if response.status_code != 200:
                response = requests.get(API_DEV, params)
    if response.status_code != 200:
        err_string = f"{API_ENDPOINT} "
        if try_dev: 
            err_string += f"and {API_DEV}"
        err_string += " unavailable."
        print(err_string)
        messages = []
    else:
        messages = response.json()
    return pd.DataFrame(messages)

In [ ]:
# df["timestamp"] = Time(df["timestamp"], format="unix_tai", scale="utc").datetime
# df.set_index("timestamp", inplace=True)
# df.index = df.index.tz_localize("UTC")

In [ ]:
# Within this time span, we need to split queries into blocks of continuous ScriptQueue uptime
# If the ScriptQueue is restarted, the salIndexes will recycle. 
# Typically this should be on the order of once a day, but let's find the boundaries using the lsst.sal.ScriptQueue.logevent_summaryState topic
topic = 'lsst.sal.ScriptQueue.logevent_summaryState'
fields = ['salIndex', 'summaryState']
dd = await efd_client.select_time_series(topic, fields, t_start, t_end)
if len(dd) == 0:
    tstops = []
    tintervals = [[t_start, t_end]]
else:
    dd['state'] = dd.apply(apply_enum, args=['summaryState', GeneralState], axis=1)
    dd['state_time'] = dd.apply(convert_index_to_time, axis=1)

    tstops = dd.query('state == "OFFLINE"').state_time.values
    #print(t_start, [e.isot for e in tstops], t_end)
    if len(tstops) == 0:
        tintervals = [[t_start, t_end]]
    if len(tstops) > 0:
        ts = tstops[0]
        ts_next = ts + TimeDelta(1 * u.second)
        tintervals = [[t_start, ts]]    
        for ts in tstops[1:]:
            tintervals.append([ts_next, ts])
            ts_next = ts + TimeDelta(1 * u.second)
        tintervals.append([ts_next, t_end])
if len(tstops) == 0:
    print(f"Found 0 ScriptQueue OFFLINE events in the time period  {t_start} to {t_end}.")
else:
    print(f"Found {len(tstops)} ScriptQueue restarts in the time period {t_start} to {t_end}, so will query in {len(tstops)+1} chunks")

In [ ]:
# Script queue --  this is where we need to find unique script salindexes in each time interval
script_stream = []
script_status = []
for tinterval in tintervals:

    # Script will find information about how scripts are configured. 
    # The description topic gives a more succinct human name to the scripts
    topic = 'lsst.sal.Script.logevent_description'
    fields = ['classname', 'description', 'salIndex']
    scriptdescription = await efd_client.select_time_series(topic, fields, tinterval[0], tinterval[1])
    scriptdescription.rename({'salIndex': 'script_salIndex'}, axis=1, inplace=True)
    scriptdescription['add_time'] = scriptdescription.apply(convert_index_to_time, axis=1)
    # This gets us more information about the script parameters, how they were configured
    topic = 'lsst.sal.Script.command_configure'
    fields = ['blockId', 'config',' executionId', 'salIndex']
    fields = await efd_client.get_fields(topic)
    fields = [f for f in fields if 'private' not in f]
    # note blockId is only filled for JSON BLOCK activities
    scriptconfig = await efd_client.select_time_series(topic, fields, tinterval[0], tinterval[1])
    scriptconfig.rename({'salIndex': 'script_salIndex'}, axis=1, inplace=True)
    scriptconfig["ts_configure_start"] = scriptconfig.apply(convert_index_to_time, axis=1)

    # Merge these together on script_salIndex which is unique over tinterval
    # Find that configuration time - script add time is << 1 second for each script and < 1 second over a night
    script_stream_t = pd.merge(scriptdescription, scriptconfig, on='script_salIndex', suffixes=['_d', '_r'])
    # Append these into a list, which we will concat after looking at all tintervals
    script_stream.append(script_stream_t)

    # The status of each of these scripts is stored in scriptQueue.logevent_script
    # so find the status of each of these scripts (this is status at individual stages).
    topic = 'lsst.sal.ScriptQueue.logevent_script'
    fields = await efd_client.get_fields(topic)
    fields = ['blockId', 'path', 'processState', 'scriptState', 'salIndex', 'scriptSalIndex', 
             'timestampProcessStart', 'timestampConfigureStart', 'timestampConfigureEnd', 'timestampRunStart', 'timestampProcessEnd' ]
    scripts = await efd_client.select_time_series(topic, fields, tinterval[0], tinterval[1])
    scripts.rename({'scriptSalIndex': 'script_salIndex'}, axis=1, inplace=True)

    # Group scripts on 'script_salIndex' to consolidate the information about its status stages
    # Make a new copy which we will fill with the max script state (== final state, given enum)
    # (new copy of this column so we don't have to deal with multi-indexes)
    scripts['finalScriptState'] = scripts['scriptState']
    script_status_t = scripts.groupby('script_salIndex').agg({'path': 'first', 'salIndex': 'max', 
                                                              'finalScriptState': 'max', 'scriptState': 'unique', 
                                                              'processState': 'unique', 
                                                              'timestampProcessStart': 'min', 
                                                              'timestampConfigureStart': 'min', 
                                                              'timestampConfigureEnd': 'max', 
                                                              'timestampRunStart': 'max', 
                                                              'timestampProcessEnd': 'max'}).sort_values(by='timestampProcessStart')
    # Convert some columns from unix timestamps to iso for readability
    # For some scripts, timestampConfigureStart is 0, but configure end is never 0
    script_status_t['ts_configure_end'] = Time(script_status_t['timestampConfigureEnd'], format='unix_tai').utc.iso
    # For some scripts, timestampRunStart is 0 (where nothing is run)
    script_status_t['ts_run_start'] = Time(script_status_t['timestampRunStart'], format='unix_tai').utc.iso
    script_status_t['ts_process_start'] = Time(script_status_t['timestampProcessStart'], format='unix_tai').utc.iso
    script_status_t['ts_process_end'] = Time(script_status_t['timestampProcessEnd'], format='unix_tai').utc.iso
    # Apply ScriptState enum for readability of final state
    script_status_t['finalScriptState'] = script_status_t.apply(apply_enum, args=['finalScriptState', ScriptState], axis=1)

    # Merge with script_stream so we get better descriptions and configuration information
    dd = pd.merge(script_stream_t, script_status_t, left_on='script_salIndex', right_index=True, suffixes=['', '_s'])
    script_status.append(dd)
    print("#info - script-status", [e.iso for e in tinterval], len(script_status), len(dd))

    
script_stream = pd.concat(script_stream)  # this may not be very useful or interesting compared to script-status df 
script_status = pd.concat(script_status)

# Create an index that will slot this into the proper place for runtime / image acquisition, etc
def find_best_script_time(x):
    # Try run start first
    best_time = x.timestampRunStart
    if best_time == 0:
        best_time = x.timestampConfigureEnd
    if best_time == 0:
        best_time = x.timestampConfigureStart
    if best_time == 0:
        best_time = x.timestampProcessStart
    return Time(best_time, format='unix_tai').utc.iso
script_status.index = script_status.apply(find_best_script_time, axis=1)
# Columns most interesting from script_status
scols = ['classname', 'description', 'config', 'script_salIndex', 'salIndex', 'blockId', 'finalScriptState', 'scriptState', 'ts_process_start', 'ts_configure_end', 'ts_run_start', 'ts_process_end']

In [ ]:
# Get error codes
topics = await efd_client.get_topics()
err_codes = [t for t in topics if 'errorCode' in t]

# will leave these for now .. should check if they are reflected in logevent_errorCode
err_topics = [t for t in topics if  'Error' in t] 

errs = []
for topic in err_codes:
    df = await efd_client.select_time_series(topic, ['errorCode', 'errorReport'], t_start, t_end)
    if len(df) > 0:
        df['topic'] = topic
        errs += [df]
if len(errs) > 0:
    errs = pd.concat(errs).sort_index()
    errs['time'] = Time(errs.index, scale='utc')
    errs.set_index('time', inplace=True)
    def strip_csc(x):
        return x.topic.replace("lsst.sal", "").replace("logevent_errorCode", "").replace(".", "") + "CSC error"
    errs['component'] = errs.apply(strip_csc, axis=1)
    # Rename some columns to match narrative log columns
    errs.rename({'errorCode': 'error_code', 'errorReport': 'message_text', 'topic': 'origin'}, axis=1, inplace=True)
    # Add a salindex so we can color-code based on this as a "source"
    errs['salIndex'] = 4

print(f"Found {len(errs)} error messages")

In [ ]:
import requests

params = {"is_human" : "either",
          "is_valid" : "true",
          "has_date_begin" : True,
          "min_date_begin" : t_start.to_datetime(),
          "max_date_begin" : t_end.to_datetime(),
          "order_by" : "date_begin",
          "limit": 10000, 
         }

API_ENDPOINT = "https://usdf-rsp.slac.stanford.edu/narrativelog/messages"

messages = usdf_requests(API_ENDPOINT, params)

if len(messages) > 0:
    def strip_rns(x):
        return x.message_text.replace("\r\n", "\n").replace("\n\n", "\n").rstrip("\n")
    def make_time(x):
        return Time(x['date_begin'], format='isot', scale='utc').iso
    def clarify_log(x):
        if x.components is None:
            component = "Log"
        else:
            component = "Log " + " ".join(x.components)
        return component
    # Strip excessive \r\n and \n\n from messages
    messages['message_text'] = messages.apply(strip_rns, axis=1)
    # Add a time index
    messages['time'] = messages.apply(make_time, axis=1)
    messages.set_index('time', inplace=True)
    # Join the components and add "Log" explicitly
    messages['component'] = messages.apply(clarify_log, axis=1)
    # rename some columns to match error data
    messages.rename({'time_lost_type': 'error_code', 'user_id': 'origin'}, axis=1, inplace=True)
    # Add a salindex so we can color-code based on this as a "source"
    messages['salIndex'] = 0

print(f"Found {len(messages)} messages in the narrative log")

In [ ]:
# Merge narrative log messages and error messages

ncols = ['component', 'origin', 'message_text', 'error_code', 'salIndex']

narrative_and_errs = []
if len(errs) > 0 and len(messages) > 0:
    print("Joined narrative log and error messages")
    narrative_and_errs = pd.concat([errs, messages]).sort_index()

elif len(errs) > 0:
    print("Error messages only; narrative log empty")
    narrative_and_errs = errs

elif len(messages) > 0:
    print("Narrative log only; error messages empty")
    narrative_and_errs = messages

# if len(joint) > 0:
#     with option_context('display.max_colwidth', None):
#         joint.style.set_properties(**{'text-align': 'left'})
#         display(HTML(joint[jcols].style.set_properties(**{'text-align': 'left'}).to_html()))

In [ ]:
# Find exposure information
topic = 'lsst.sal.CCCamera.logevent_endOfImageTelemetry' 
fields = ['imageName', 'imageIndex', 'exposureTime', 'darkTime', 'measuredShutterOpenTime', 'additionalValues', 'timestampAcquisitionStart', 'timestampDateEnd', 'timestampDateObs']
image_acquisition = await efd_client.select_time_series(topic, fields, t_start, t_end)
image_acquisition['ts_process_start'] = Time(image_acquisition['timestampAcquisitionStart'], format='unix_tai').utc.iso
image_acquisition['ts_process_end'] = Time(image_acquisition['timestampDateEnd'], format='unix_tai').utc.iso
image_acquisition['ts_run_start'] = Time(image_acquisition['timestampDateObs'], format='unix_tai').utc.iso
image_acquisition['salIndex'] = -1
image_acquisition['script_salIndex'] = 0
image_acquisition['finalStatus'] = "Image Acquired"
def make_config_col_for_image(x):
    return f"exp {x.exposureTime} // dark {x.darkTime} // open {x.measuredShutterOpenTime} "
image_acquisition['config'] = image_acquisition.apply(make_config_col_for_image, axis=1)
image_acquisition.index = image_acquisition['ts_process_start'].copy()

In [ ]:
# from lsst.summit.utils import ConsDbClient
# # Not sure of summit consdb access, just use USDF for now
# os.environ["LSST_CONSDB_PQ_URL"] = "http://consdb-pq.consdb:8080/consdb"
# os.environ["no_proxy"] += ",.consdb"

# day_obs_int_min = int(day_obs_min.replace('-', ''))
# day_obs_int_max = int(day_obs_max.replace('-', ''))

# # Use the ConsDB Client, and add a couple of tries 
# consdb = ConsDbClient()

# instrument = 'lsstcomcam'
# visit_query = f'''
#     SELECT * 
#     FROM cdb_{instrument}.visit1
#      WHERE day_obs >= {day_obs_int_min}
#      and day_obs  <= {day_obs_int_max}
# '''
# try:
#     visits = consdb.query(visit_query).to_pandas()
# except requests.HTTPError or requests.JSONDecodeError:
#     # Try twice
#     visits = consdb.query(visit_query).to_pandas()
    
# visits.set_index('visit_id', inplace=True)
# (Time(visits.query('exposure_name == "CC_O_20241115_000347"')['exp_midpt'].values[0], format='isot', scale='tai') - TimeDelta(15 * u.second)).utc.iso

In [ ]:
print(f"Found {len(image_acquisition)} exposures from the CCCamera endOfImageTelemetry")

# exposure log too
min_dayobs_int = int(t_start.iso[0:10].replace('-', ''))
max_dayobs_int = int(t_end.iso[0:10].replace('-', ''))
params = {"is_human" : "either",
          "is_valid" : "true",
          "min_day_obs" : min_dayobs_int,
          "max_day_obs" : max_dayobs_int,
          "limit": 10000, 
         }

API_ENDPOINT = "https://usdf-rsp.slac.stanford.edu/exposurelog/messages"

exp_logs = usdf_requests(API_ENDPOINT, params)
print(f"Found {len(exp_logs)} messages in the exposure log")

# Add exposure log
if len(exp_logs) > 0:
    # Note that we don't have a time in the exposure log - just a visit id, to join against the image acquisition information
    exp = pd.merge(image_acquisition, exp_logs, how='right', left_on='imageName', right_on='obs_id')
    # Set the time for the exposure log just slightly after the image start time
    exp_log_image_time = Time(exp['timestampAcquisitionStart'], format='unix_tai') + TimeDelta(0.01 * u.second)
    exp_logs['img_time'] = exp_log_image_time.utc.iso
    exp_logs.set_index('img_time', inplace=True)
    exp_logs['salIndex'] = 0
    exp_logs['script_salIndex'] = 0
    # Rename some columns now so that we can consolidate them here
    exp_logs.rename({'obs_id': 'imageName', 'user_id': 'config', 'message_text': 'additionalValues', 'exposure_flag': 'finalStatus'}, axis=1, inplace=True)
    image_and_logs = pd.concat([image_acquisition, exp_logs]).sort_index()
    print("Joined exposure and exposure log")
else:
    image_and_logs = image_acquisition


icols = ['imageName', 'additionalValues', 'exposureTime', 'darkTime', 'measuredShutterOpenTime', 'finalStatus', 'script_salIndex', 'salIndex', 'ts_process_start', 'ts_run_start', 'ts_process_end']

In [ ]:
# Now rename columns so we can put these all into the same dataframe
# goal columns : 
cols = ['time', 'name', 'description', 'config', 'script_salIndex', 'salIndex', 'finalStatus', 'ts_process_start', 'ts_configure_end', 'ts_run_start', 'ts_process_end'] 

# columns from scripts
script_cols = ['classname', 'description', 'config', 'script_salIndex', 'salIndex', 'blockId', 'finalScriptState', 'scriptState', 'ts_process_start', 'ts_configure_end', 'ts_run_start', 'ts_process_end']
script_status.rename({'classname': 'name', 'finalScriptState': 'finalStatus'}, axis=1, inplace=True)
# columns from narrative and errors
narrative_cols = ['component', 'origin', 'message_text', 'error_code', 'salIndex']
narrative_and_errs.rename({'component': 'name', 'origin': 'config', 'message_text': 'description', 'error_code': 'script_salIndex'}, axis=1, inplace=True)
narrative_and_errs['ts_process_start'] = narrative_and_errs.apply(convert_index_to_time, axis=1)
# columns from images_and_logs
image_cols = ['imageName', 'additionalValues', 'config', 'finalStatus', 'script_salIndex', 'salIndex', 'ts_process_start', 'ts_run_start', 'ts_process_end']
image_and_logs.rename({'imageName': 'name', 'additionalValues' : 'description'}, axis=1, inplace=True) 


efd_and_messages = pd.concat([script_status, narrative_and_errs, image_and_logs]).sort_index()
# Wrap description, which can may have long zero-space messages in the errors
efd_and_messages['description'] = efd_and_messages['description'].str.wrap(100)
# use an integer index, which makes it easier to pull up values plus avoids occasional failures of time uniqueness
efd_and_messages.reset_index(drop=False, inplace=True)
efd_and_messages.rename({'index': 'time'}, axis=1, inplace=True)


# DROP SOME MESSAGES
print("Dropping CBP messages and error messages from ElectrometerCSC and TuneableLaserCSC")
efd_and_messages = efd_and_messages.query('~ (config.str.contains("component: CBP") or config.str.contains("CBP,"))')
efd_and_messages = efd_and_messages.query('~ (name.str.contains("ElectrometerCSC error") or name.str.contains("TunableLaserCSC error"))')


def highlight_salindex(s):
    # Colors from https://medialab.github.io/iwanthue/
    if s.salIndex == 0:     # narrative log
        return ['background-color: #cf7ddc'] * len(s)
    elif s.salIndex ==  4:  # error messages
        return ['background-color: #9cb5d5'] * len(s)
    elif s.salIndex == 1:   # simonyi queue
        return ['background-color: #b4c546'] * len(s)
    elif s.salIndex == 2:   # aux tel queue
        return ['background-color: #bab980'] * len(s)
    elif s.salIndex == 3:   # ocs queue
        return ['background-color: #b2baad'] * len(s)
    #elif s.salIndex == -1:  # image
    #    return ['background-color: #b6ecf5'] * len(s)
    else:
        return [''] * len(s)

print(f"Total combined messages {len(efd_and_messages)}")
print("Color coding by salIndex (1/2/3 scriptqueue index) + data source (narrative or exposure log, or EFD logevent_errorCode messages)")
print('')

with option_context('display.max_colwidth', 0):
    display(HTML(efd_and_messages[cols].style.apply(highlight_salindex, axis=1).set_table_styles([dict(selector='th', props=[('text-align', 'left')])]).set_properties(**{'text-align': 'left'}).to_html()))